In [1]:
%load_ext autoreload
%autoreload 2
    

# Running MD simulations using nff and ASE

This Jupyter Notebook shows how the `nff` package interfaces with the Atomistic Simulation Environment (ASE). We assume the user went through tutorial `01_training`, so we can load the pretrained models without having to train them again.

As before, importing the dependencies:

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from ase import Atoms
from ase.md.verlet import VelocityVerlet

from nff.md.nve import Dynamics
from nff.data import Dataset
from nff.train import load_model, evaluate
import nff.utils.constants as const
from ase import units
from nff.io import NeuralFF, AtomsBatch

## Loading the relevant data

We reload the dataset and create a `GraphLoader` as we did last time:

In [3]:
dataset = Dataset.from_file('data/dataset.pth.tar')


### Creating Atoms

As before, we can create an `Atoms` object from any element of the dataset. Let's take the first one, for simplicity:

In [4]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   )

### Creating the ASE calculator

Now we just have to load the ASE calculator from a pretrained model. One way of doing so is through the in-build `from_file` method. You just have to specify the folder where the model was trained and subsequently stored.

In [5]:
nff_ase = NeuralFF.from_file('sandbox/', device=0)

/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'nff.nn.models.schnet.SchNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve 

Assigning this calculator to `atoms` is easy:

In [6]:
atoms.set_calculator(nff_ase)

### Configuring the dynamics for the system

In this example, we will run an NVE dynamics simulation. We will use the default parameters there implemented to run a trajectory for an ethanol molecule. The parameters we will specify are the following:

* `T_init`: initial temperature of the simulation
* `time_step`: time step in femtoseconds
* `thermostat`: ASE integrator to use when performing the simulation
* `thermostat_params`: keyword arguments for ase.Integrator class, will be different case-by-case
* `steps`: number of steps to simulate
* `save_frequency`: how often (in steps) save the pose of the molecule in a file
* `nbr_list_update_freq`: how often (in steps) to update the neighbor list (not yet implemented)
* `thermo_filename`: output file for the thermodynamics log
* `traj_filename`: output file for the ASE trajectory file
* `skip`: number of initial frames to skip when recording the trajectory

In [7]:
md_params = {
    'T_init': 450,
    'time_step': 0.5,
#     'thermostat': NoseHoover,   # or Langevin or NPT or NVT or Thermodynamic Integration
#     'thermostat_params': {'timestep': 0.5 * units.fs, "temperature": 120.0 * units.kB,  "ttime": 20.0}
    'thermostat': VelocityVerlet,  
    'thermostat_params': {'timestep': 0.5 * units.fs},
    'steps': 200,
    'save_frequency': 10,
    'nbr_list_update_freq': 20,
    'thermo_filename': 'thermo.log',
    'traj_filename': 'atoms.traj',
    'skip': 0
}

In [8]:
nve = Dynamics(atoms, md_params)
nve.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.4912      -0.1916       0.6827   586.9



/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/ase/atoms.py:921: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  np.VisibleDeprecationWarning)
/home/saxelrod/Repo/projects/master/NeuralForceField/nff/nn/utils.py:120: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  nbr_list = mask.nonzero()


0.0050           0.4907      -0.1218       0.6126   526.6

0.0100           0.4907      -0.2675       0.7581   651.7

0.0150           0.4931      -0.0414       0.5345   459.5

0.0200           0.4931      -0.0746       0.5677   488.0

0.0250           0.4939      -0.0384       0.5323   457.6

0.0300           0.4920      -0.0283       0.5203   447.2

0.0350           0.4925       0.0460       0.4465   383.8

0.0400           0.4922      -0.0920       0.5842   502.1

0.0450           0.4925       0.0348       0.4577   393.4

0.0500           0.4930      -0.0101       0.5031   432.4

0.0550           0.4935      -0.0357       0.5291   454.8

0.0600           0.4918      -0.0757       0.5675   487.8

0.0650           0.4905      -0.0489       0.5394   463.7

0.0700           0.4919       0.0108       0.4811   413.5

0.0750           0.4921      -0.0411       0.5333   458.4

0.0800           0.4934       0.1537       0.3398   292.1

0.0850           0.4937      -0.0264       0.5201   447.

### Models with directed neighbor lists

The default assumes that you're using SchNet, which uses an undirected neighbor list to save memory. If you're using Painn, DimeNet, or any model with directional information, you will need a directed neighbor list. If you don't specify this then you will get an error. 

While these models automatically make any neighbor list into a directed one when called, the same is not true of periodic boundary conditions. The dataest offsets from the PBC depend on whether the dataset neighbor list is directed or not.

In [9]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   )
nff_ase = NeuralFF.from_file('sandbox_painn/', device=0)
atoms.set_calculator(nff_ase)

try:
    nve = Dynamics(atoms, md_params)
    nve.run()
except Exception as e:
    print(e)

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
Painn needs a directed neighbor list


If you do then you will be fine!

In [10]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   directed=True
                   )
nff_ase = NeuralFF.from_file('sandbox_painn/', device=0)
atoms.set_calculator(nff_ase)

nve = Dynamics(atoms, md_params)
nve.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.3289      -0.1615       0.4904   421.5

0.0050           0.3281      -0.0909       0.4190   360.1

0.0100           0.3284      -0.2482       0.5766   495.6

0.0150           0.3281      -0.0493       0.3774   324.5

0.0200           0.3307       0.0222       0.3085   265.2

0.0250           0.3314      -0.1050       0.4364   375.1

0.0300           0.3315       0.0617       0.2698   231.9

0.0350           0.3285      -0.0881       0.4166   358.1

0.0400           0.3284      -0.3012       0.6296   541.2

0.0450           0.3273      -0.2460       0.5734   492.9

0.0500           0.3282      -0.0552       0.3834   329.6

0.0550           0.3309      -0.0851       0.4160   357.6

0.0600           0.3311      -0.2226       0.5536   475.9

0.0650           0.3326       0.0409       0.2917   250.8

0.0700           0.3326      -0.0412       0.3737   321.2

0.0750           0.3304      -0.0243       0.3547   304.9


The dynamics conserved the energy. The temperature varied throughout the simulation, as expected.

# Additional properties

You can also specify the properties that you want the calculator to predict. The default is `energy` and `forces`, but you can also add `stress`.

However, if you run this for ethanol you will get an error, because it doesn't have any lattice vectors:

In [11]:
nff_ase.properties = ['energy', 'forces', 'stress']
atoms.set_calculator(nff_ase)

try:
    nve = Dynamics(atoms, md_params)
    nve.run()
except Exception as e:
    print(e)

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.4323      -0.1401       0.5723   492.0

You have 0 lattice vectors: volume not defined


If we make an artificial unit cell, it all works fine:

In [12]:
nff_ase.properties = ['energy', 'forces', 'stress']
atoms.set_calculator(nff_ase)
atoms.set_cell(2 * np.identity(3))
nve = Dynamics(atoms, md_params)
nve.run()



Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.4550      -0.1566       0.6116   525.7

0.0050           0.4557       0.0643       0.3914   336.5

0.0100           0.4552      -0.0342       0.4894   420.7

0.0150           0.4569      -0.0333       0.4902   421.4

0.0200           0.4556      -0.0448       0.5003   430.1

0.0250           0.4554      -0.0607       0.5160   443.6

0.0300           0.4555       0.0376       0.4179   359.2

0.0350           0.4535      -0.1467       0.6002   515.9

0.0400           0.4537      -0.2165       0.6702   576.1

0.0450           0.4550      -0.0728       0.5278   453.7

0.0500           0.4545      -0.1732       0.6277   539.5

0.0550           0.4554      -0.1396       0.5949   511.4

0.0600           0.4562      -0.0454       0.5016   431.2

0.0650           0.4553      -0.0909       0.5462   469.5

0.0700           0.4556      -0.0788       0.5344   459.3

0.0750           0.4546      -0.2364       0.6910   594.0


Since `stress` is an implemented property, we can get the stress on the atoms:

In [13]:
atoms.get_stress()

array([0.27654982, 0.19260846, 0.5454004 , 0.23866758, 0.10975268,
       0.25836495], dtype=float32)

## Visualizing the trajectory

To visualize the trajectory in this Jupyter Notebook, you will have to install the package [nglview](https://github.com/arose/nglview).

In [14]:
import nglview as nv
from ase.io import Trajectory

Displaying the trajectory:

In [15]:
%matplotlib notebook

traj = Trajectory('atoms.traj')
nv.show_asetraj(traj)

NGLWidget(max_frame=20)

Looks like the atoms are still together. Visual inspection says that the trajectory is reasonable. Yay for `nff`!